# VM Management — Title / Objective

**Objective:** Implement VM creation and management using libvirt and Python bindings, or simulate VM scheduling when libvirt is unavailable.

Run the single code cell below in Google Colab. The notebook will attempt to use libvirt; if unavailable it will run a local simulation.

In [ ]:

import time, random, uuid
try:
    import libvirt
    HAVE_LIBVIRT = True
except:
    HAVE_LIBVIRT = False

hosts = {
    "host-A": {"cpu": 8, "mem": 16384, "vms": []},
    "host-B": {"cpu": 4, "mem": 8192, "vms": []},
    "host-C": {"cpu": 6, "mem": 12288, "vms": []}
}

vms = {}

def now(): return time.strftime("%H:%M:%S")

def fit_host_for_vm(cpu, mem):
    candidates = []
    for h, info in hosts.items():
        used_cpu = sum(v["cpu"] for v in info["vms"])
        used_mem = sum(v["mem"] for v in info["vms"])
        if used_cpu + cpu <= info["cpu"] and used_mem + mem <= info["mem"]:
            candidates.append((h, info["cpu"] - used_cpu))
    if not candidates: return None
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[0][0]

def simulate_create_vm(name, cpu, mem):
    hid = fit_host_for_vm(cpu, mem)
    if not hid:
        print(f"[{now()}] No host has capacity for VM {name} ({cpu}c, {mem}MB)")
        return None
    vm_id = str(uuid.uuid4())[:8]
    vm = {"id": vm_id, "name": name, "cpu": cpu, "mem": mem, "host": hid, "state": "running"}
    vms[vm_id] = vm
    hosts[hid]["vms"].append(vm)
    print(f"[{now()}] Created VM {name} id={vm_id} on host={hid}")
    return vm

def simulate_delete_vm(vm_id):
    vm = vms.get(vm_id)
    if not vm:
        print(f"[{now()}] VM {vm_id} not found")
        return False
    host = vm["host"]
    hosts[host]["vms"] = [x for x in hosts[host]["vms"] if x["id"] != vm_id]
    del vms[vm_id]
    print(f"[{now()}] Deleted VM id={vm_id} from host={host}")
    return True

def simulate_migrate_vm(vm_id, dest_host):
    vm = vms.get(vm_id)
    if not vm:
        print(f"[{now()}] VM {vm_id} not found")
        return False
    if dest_host not in hosts:
        print(f"[{now()}] Destination host {dest_host} unknown")
        return False
    used_cpu = sum(v["cpu"] for v in hosts[dest_host]["vms"])
    used_mem = sum(v["mem"] for v in hosts[dest_host]["vms"])
    if used_cpu + vm["cpu"] > hosts[dest_host]["cpu"] or used_mem + vm["mem"] > hosts[dest_host]["mem"]:
        print(f"[{now()}] Not enough capacity on {dest_host} to migrate VM {vm_id}")
        return False
    src = vm["host"]
    hosts[src]["vms"] = [x for x in hosts[src]["vms"] if x["id"] != vm_id]
    vm["host"] = dest_host
    hosts[dest_host]["vms"].append(vm)
    print(f"[{now()}] Migrated VM id={vm_id} from {src} to {dest_host}")
    return True

def list_hosts():
    for h, info in hosts.items():
        used_cpu = sum(v["cpu"] for v in info["vms"])
        used_mem = sum(v["mem"] for v in info["vms"])
        print(f"{h}: CPU {used_cpu}/{info['cpu']} MEM {used_mem}/{info['mem']} VMs={len(info['vms'])}")

def list_vms():
    for vm in vms.values():
        print(f"id={vm['id']} name={vm['name']} host={vm['host']} cpu={vm['cpu']} mem={vm['mem']} state={vm['state']}")

def try_libvirt_list_domains():
    if not HAVE_LIBVIRT:
        print(f"[{now()}] libvirt not available; running simulation")
        return
    try:
        conn = libvirt.open(None)
        if conn is None:
            print(f"[{now()}] Cannot open libvirt connection")
            return
        doms = conn.listAllDomains()
        print(f"[{now()}] libvirt domains:")
        for d in doms:
            print(d.name(), d.ID(), d.isActive())
        conn.close()
    except Exception as e:
        print(f"[{now()}] libvirt error: {e}")

def sample_run():
    print(f"[{now()}] Sample VM scheduling simulation start")
    for i in range(6):
        simulate_create_vm(f"vm-{i+1}", cpu=random.choice([1,2,4]), mem=random.choice([512,1024,2048]))
        time.sleep(0.2)
    list_hosts()
    print("--- Attempt migration ---")
    ids = list(vms.keys())
    if ids:
        simulate_migrate_vm(ids[0], "host-B")
    list_hosts()
    print("--- Delete one VM ---")
    if ids:
        simulate_delete_vm(ids[-1])
    list_vms()
    print(f"[{now()}] Sample run complete")

if __name__ == "__main__":
    try_libvirt_list_domains()
    sample_run()
